In [ ]:
import sys
sys.path.append('../../')
%load_ext autotime

# Introduction
In this notebook, we are numerically calculating the average Hamiltonian produced by the pulse scheme proposed in [1] to mix a native XY Heisenberg Hamiltonian (symmetric exchange) into a new XY Hamiltonian with anti-symmetric exchange (the z-component of the DM interaction).


1. Nishad, N., Keselman, A., Lahaye, T., Browaeys, A. & Tsesses, S. Quantum simulation of generic spin-exchange models in Floquet-engineered Rydberg-atom arrays. Phys. Rev. A 108, 053318 (2023).


In [ ]:
from models.spin_chain import LatticeGraph, DiagonEngine
import numpy as np
from scipy.linalg import expm
from __future__ import annotations

# Local Rotations
This paper uses local rotations in a size 4 unit cell (PBC, a ring of atoms L = 4N). Figure 2 in the paper has a good illustration. There are two basic pulses that rotate different sites with different amounts of phase to turn on/off the positive/negative DM/XY interactions. The evolution times between the local phase rotations determine the nature (mostly XY or mostly DM) of the average Hamiltonian in one cycle.

For now, we want to ignore the effect of finite pulse durations, so these functions return a total phase of rotation (an integrated delta function pulse) rather than a frequency/energy to be integrated over a finite time. We define another function that turns off the native Hamiltonian during these phase rotations (delta function pulses).

In [ ]:
def DM_z_period4(t, i):
    phase = np.pi / 2 * (i % 4)
    if t == "+DM":
        return phase/2
    elif t == "-DM":
        return -phase/2
    else:
        return 0

def XY_z_period4(t, i):
    phase = np.pi - 3. * np.pi / 2 * (i % 4)
    if t == "+XY":
        return phase/2
    elif t == "-XY":
        return -phase/2
    else:
        return 0

def native(t, i, j):
    if t in ["+DM", "-DM", "+XY", "-XY"]:
        return 0
    else:
        return J/2

# Define the Hamiltonian
This is simply a list of terms in the (parametrized) Hamiltonian we want to numerically evolve in time. The first entry of each term is the operator as a string. The second element is the "strength" of the interaction, which here is parametrized by pulse type (as above) to make a piece-wise defined Hamiltonian (though one could consider defining a time-continuous parametrization, and we might do this later). The last element is the "connectivity" or "range" of the interactions defined either by a specific string or an inverse range value (alpha = 3 would be dipolar range, alpha = np.inf is an on-site interaction).

Here we initially consider the smallest system this pulse scheme allows, 4 atoms in a ring.

In [ ]:
XY_terms = [['XX', native, 'nn'], ['yy', native, 'nn'],
             ['z', DM_z_period4, np.inf], ['z', XY_z_period4, np.inf]]
XY_graph = LatticeGraph.from_interactions(4, XY_terms, pbc=True)

## Define the Pulse Sequence
Here, the timing of the pulse sequence is defined. The paramList selects which pulses we want from our parametrization above, and dtList defines how long each of those pieces is evolved in time. Delta pulses act for zero time, so the only time evolution that happens in this example are the "free-evolution" times under the native Hamiltonian.

In [ ]:
tD = 1000e-9
tJ = 100e-9
tmJ = tJ
J = 10
paramList = ["nat", "+DM", "nat", "+XY", "nat", "-XY", "nat", "-DM", "nat"]
dtList = [tJ, 0, tD, 0, 2 * tmJ, 0, tD, 0, tJ]

This is our target Hamiltonian.

In [ ]:
DM_terms = [['xy', 1/2, 'nn'], ['yx', -1/2, 'nn']]
DM_graph = LatticeGraph.from_interactions(4, DM_terms, pbc=True)

Under the hood, we use QuSpin to perform the time-evolution and calculate the Floquet/Average Hamiltonian. We compute the Hilbert-Schmidt overlap of the relevant Hamiltonians.

In [ ]:
computation = DiagonEngine(XY_graph, unit_cell_length=4)
HF = computation.get_quspin_floquet_hamiltonian(paramList, dtList)
# print(HF)
H_XY = computation.get_quspin_hamiltonian(0)/J
XY_frobenius_loss = computation.frobenius_loss(HF, H_XY)
H_DM = DiagonEngine(DM_graph, unit_cell_length=4).get_quspin_hamiltonian(0)
DM_frobenius_loss = computation.frobenius_loss(HF, H_DM)
print("XY Frobenius loss:", XY_frobenius_loss)
print("DM Frobenius loss:", DM_frobenius_loss)

We can compute another type of fidelity metric (used in the referenced paper) that approaches zero as the unitary evolution computed from the Hamiltonians approach perfect overlap. It is a much more sensitive measure of overlap than the Hilbert-Schmidt overlap we've defined above. This is probably a better metric for numerical optimization of the pulse sequence and therefore likely why it was the metric of choice in the reference paper.

In [ ]:
T = sum(dtList) # the Floquet period
DM_norm_loss = computation.norm_identity_loss(HF, H_DM)
XY_norm_loss = computation.norm_identity_loss(HF, H_XY)
print("XY loss:", XY_norm_loss)
print("DM loss:", DM_norm_loss)

How should we think about the triangle of norm losses between each of the three Hamiltonians

In [ ]:
cross_frobenius_loss = computation.frobenius_loss(H_DM, H_XY)
print("cross Frobenius loss (should be 1):", cross_frobenius_loss)
cross_norm_loss = computation.norm_identity_loss(H_DM, H_XY)
print("norm loss (should be large):", cross_norm_loss)

# Open Chain DM Exchange

In [ ]:
XY_terms = [['XX', native, 'nn'], ['yy', native, 'nn'],
             ['z', DM_z_period4, np.inf], ['z', XY_z_period4, np.inf]]
XY_graph = LatticeGraph.from_interactions(4, XY_terms)

In [ ]:
DM_terms = [['xy', 1/2, 'nn'], ['yx', -1/2, 'nn']]
DM_graph = LatticeGraph.from_interactions(4, DM_terms)